# Day 10
Find the description of the problem [here](https://adventofcode.com/2025/day/10)!

## Part 1

Puzzle input:

In [212]:
with open("input_files/day_10.txt") as input_file:
    input = input_file.read()

Test input:

In [213]:
# # Comment this cell to use the puzzle input instead of the test input
# input = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
# [...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
# [.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"""

Parse the input:

In [214]:
target_list = []
buttons_list = []
requirements_list = []

for line in input.splitlines():
    line_split = line.split(" ")
    
    # Indicators
    indicators_line = []
    for indicator in line_split[0][1:-1]:
        if indicator == "#":
            indicators_line.append(1)
        else:
            indicators_line.append(-1)
    target_list.append(indicators_line)

    # Buttons
    buttons_line = []
    for button in line_split[1:-1]:
        buttons_line.append(list(map(int, button[1:-1].split(","))))
    buttons_list.append(buttons_line)
    
    # Joltage requirements
    requirements_list.append(list(map(int, line_split[-1][1:-1].split(","))))

Define a quick function to push a button and modify the indicators:

In [215]:
def push_button(indicator, button):
    for i in button:
        indicator[i] *= -1

Pushing a button twice is the same as not pushing it at all, so each button should only be pressed once. We can iterate through all the button combinations easily:

In [216]:
from itertools import product

presses_required = 0
for target, buttons in zip(target_list, buttons_list):
    starting_indicators = [-1] * len(target)
    correct_combinations = []
    for combination in product([0, 1], repeat=len(buttons)):
        indicators = starting_indicators.copy()
        for i, push in enumerate(combination):
            if push:
                push_button(indicators, buttons[i])
        if indicators == target:
            correct_combinations.append(sum(combination))
    presses_required += min(correct_combinations)
print(f"The fewest button presses required to correctly configure the indicator lights is {presses_required}.")

The fewest button presses required to correctly configure the indicator lights is 517.


## Part 2

This is a mathematical problem and not a code problem (I'm sure it can be coded, but I don't even know where to start). First of all, we convert the buttons to vectors representing which joltage level counters are affected:

In [217]:
import numpy as np

buttons_list_vectorized = []
requirements_list_vecorized = []
for buttons, requirements in zip(buttons_list, requirements_list):
    buttons_vectorized = []
    for button in buttons:
        new_button = [0] * len(requirements)
        for i in button:
            new_button[i] = 1
        buttons_vectorized.append(np.array(new_button))
    buttons_list_vectorized.append(buttons_vectorized)
    requirements_list_vecorized.append(np.array(requirements))

Now it's a matter of solving the mathematical problem. We need to solve the following formula: $a_1 x_1 + a_2 x_2 + \cdots + a_n x_n = y$, where $y$ is the joltage requirement, $x_n$ are the buttons, and $a_n$ is the amount of presses for each button, which must not be negative. Finally, the objective is to minimize $a_1 + a_2 + \cdots + a_n$ with these constraints.

I looked online how to solve ~~linear~~ integer algebra problems and found the `pulp` library.

In [ ]:
import pulp

presses_required = 0
for buttons, requirements in zip(buttons_list_vectorized, requirements_list_vecorized):
    X = np.column_stack(buttons)
    m, n = X.shape

    # Create a Linear Programming Problem
    problem = pulp.LpProblem("MinSum", pulp.LpMinimize)

    # Define the non-negative integer variables whose sum must be minimized and add them to the problem
    button_presses = [pulp.LpVariable(f"a_{i}", lowBound=0, cat="Integer") for i in range(n)]
    problem += pulp.lpSum(button_presses)

    # Add to the problem that the sum must equal the requirements
    for row in range(m):
        problem += pulp.lpSum(X[row, j] * button_presses[j] for j in range(n)) == requirements[row]

    # Solve the problems onto the a variables
    problem.solve(pulp.PULP_CBC_CMD(msg=False))
    result = [press.value() for press in button_presses]
    presses_required += sum(result)
print(f"The fewest button presses required to correctly configure the joltage level counters is {int(presses_required)}.")

The fewest button presses required to correctly configure the joltage level counters is 21469.
